In [1]:
# 大紀元
# 爬蟲套件
import requests
from bs4 import BeautifulSoup
import json
from fake_useragent import UserAgent
# 資料整理類套件
import pandas as pd
import re
import time
import random
import datetime
from tomorrow import threads
from pymongo import MongoClient
from datetime import datetime
import pymongo

In [2]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [18]:
def GetAllLinks():
    '''最後一頁233'''
    links = []
    page = 1
    while True:
        url = "https://www.epochtimes.com/b5/ncid1077884_"+str(page)+".htm"
        res = requests.get(url,headers=headers)
        soup = BeautifulSoup(res.text)
        try:
            for i in soup.find("div",class_="posts-list").find_all("div",class_="title"):
                links.append(i.find("a").get("href"))
        except AttributeError:
            break
        if page > 2:
            break
        page += 1
    return links

In [11]:
def GetNews(response):
    '''大紀元內容爬取'''
    soup = BeautifulSoup(response.text)
    Url = soup.find("div",class_="large-9 medium-9 small-12 column").find("a",class_="djy-fonts icons").get("href")
    Title = soup.find("h1",class_="blue18 title").text
    CreateTime = soup.find("div",class_="mbottom10 large-12 medium-12 small-12 columns").find("time").text.replace("\t","").replace("更新: ","")
    CreateTime = datetime.strptime(CreateTime,'%Y-%m-%d %I:%M %p')
    list_content = []
    for i in soup.find("div",id="artbody").find_all("p"):
        list_content.append(i.text)
    Content = "".join(list_content)
    Tag = []
    for i in soup.find("div",class_="mbottom10 large-12 medium-12 small-12 columns").find_all("a"):
        Tag.append(i.text)
    if len(soup.find('div',id='artbody').find_all('p')[-1].text) < 10:
        Author = soup.find('div',id='artbody').find_all('p')[-1].text.replace("責任編輯：","")
    else:
        Author = "無作者"

    ndf = pd.DataFrame(data = [{"Source":"News",
                                "Site":"大紀元",
                                "Url":Url,
                                "Title":Title,
                                "Author":Author,
                                "CreateTime":CreateTime,
                                "Content":Content,
                                "Tag":Tag}],
                      columns = ['Source', 'Site', 'Url', 'CreateTime', 'Author', 'Title', 'Content','Tag'])
    print(Url)
    return ndf

In [12]:
# # 測試內容
url = "https://www.epochtimes.com/b5/19/11/23/n11675628.htm"
res = requests.get(url,headers=headers)
GetNews(res)

https://www.epochtimes.com/gb/19/11/23/n11675628.htm


,Source,Site,Url,CreateTime,Author,Title,Content,Tag
0,News,大紀元,https://www.epochtimes.com/gb/19/11/23/n116756...,2019-11-23 14:27:00,陳安,蔡英文：中共介入選舉明顯 民眾應戒慎恐懼,【大紀元2019年11月23日訊】澳洲媒體報導，中共間諜自陳滲透台灣試圖干預選舉；總統蔡英文...,"[蔡英文, 2020總統大選, 間諜, 李晏榕, 王立強]"


In [13]:
# 多線程爬蟲定義
@threads(100)
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

In [14]:
def CrawlAllNews(links):
    # 爬取全部連結的新聞內容
    print('There are {} links in pages.'.format(len(links)))
    
    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    allData = []
    for response in responses:
        try:
            ndf = GetNews(response)
            allData.append(ndf)
        except:
            pass
    df = pd.concat(allData, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df

In [15]:
def dataframe_to_mongo_test(select_df):
    user_name = "test"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "test"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.test   #DB_name
    collection = db.test_news  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")


In [16]:
def dataframe_to_mongo_political(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")
   

In [19]:
links = GetAllLinks() 
df = CrawlAllNews(links)

There are 90 links in pages.
https://www.epochtimes.com/gb/19/12/11/n11715995.htm
https://www.epochtimes.com/gb/19/12/11/n11715991.htm
https://www.epochtimes.com/gb/19/12/11/n11716017.htm
https://www.epochtimes.com/gb/19/12/11/n11716078.htm
https://www.epochtimes.com/gb/19/12/11/n11715920.htm
https://www.epochtimes.com/gb/19/12/11/n11716001.htm
https://www.epochtimes.com/gb/19/12/11/n11715986.htm
https://www.epochtimes.com/gb/19/12/11/n11716006.htm
https://www.epochtimes.com/gb/19/12/11/n11715739.htm
https://www.epochtimes.com/gb/19/12/11/n11715901.htm
https://www.epochtimes.com/gb/19/12/11/n11715500.htm
https://www.epochtimes.com/gb/19/12/11/n11715824.htm
https://www.epochtimes.com/gb/19/12/11/n11715826.htm
https://www.epochtimes.com/gb/19/12/11/n11715758.htm
https://www.epochtimes.com/gb/19/12/11/n11715720.htm
https://www.epochtimes.com/gb/19/12/11/n11715496.htm
https://www.epochtimes.com/gb/19/12/11/n11715419.htm
https://www.epochtimes.com/gb/19/12/11/n11715361.htm
https://www.epoch

In [ ]:
dataframe_to_mongo_political(df)

In [20]:
df

,Source,Site,Url,CreateTime,Author,Title,Content,Tag
0,News,大紀元,https://www.epochtimes.com/gb/19/12/11/n117159...,2019-12-12 00:39:00,玉珍,綠黨民調：近7成台灣民眾對中共反感,【大紀元2019年12月11日訊】（大紀元記者吳旻洲台灣台北報導）2020總統、立委選舉進入...,"[綠黨, 民調, 蔡英文, 韓國瑜, 中共]"
1,News,大紀元,https://www.epochtimes.com/gb/19/12/11/n117159...,2019-12-12 00:35:00,陳玟綺,中共否認介入台大選 陸委會：事實勝於雄辯,【大紀元2019年12月11日訊】（大紀元記者李怡欣台灣台北報導）國際多次提醒中共意圖干預台...,"[對台26條, 台灣大選, 陸委會, 國台辦]"
2,News,大紀元,https://www.epochtimes.com/gb/19/12/11/n117160...,2019-12-11 22:11:00,芸清,再爆李家砂石不繳補償金 韓辦：黃國昌甘心淪民進黨「分機」,【大紀元2019年12月11日訊】（大紀元記者李怡欣台北報導）關於時代力量立委黃國昌11日再...,"[黃國昌, 李佳芬, 韓國瑜, 大通砂石]"
3,News,大紀元,https://www.epochtimes.com/gb/19/12/11/n117160...,2019-12-11 21:40:00,陳安,台灣醫界聯盟基金會：醫院應拒絕政治人物作秀,【大紀元2019年12月11日訊】台灣教授協會、台灣醫界聯盟基金會今天召開記者會，呼籲落實檢...,"[台大醫院, 國民黨, 陳玉珍, 台灣教授協會, 台灣醫界聯盟基金會]"
4,News,大紀元,https://www.epochtimes.com/gb/19/12/11/n117159...,2019-12-11 21:40:00,陳安,不分區參選爆炸 8政黨電視辯論爭取選民認同,【大紀元2019年12月11日訊】2020年立委選舉小黨林立，共有19個政黨提出不分區名單；...,"[民視, 不分區立委, 公督盟, 電視辯論, 公民監督國會聯盟]"
5,News,大紀元,https://www.epochtimes.com/gb/19/12/11/n117160...,2019-12-11 21:31:00,文正,避免經濟鎖國 韓辦：重新協商ECFA,【大紀元2019年12月11日訊】（大紀元記者李怡欣台灣台北報導）韓國瑜競選辦公室11日發布...,"[韓國瑜, ECFA, FTA]"
6,News,大紀元,https://www.epochtimes.com/gb/19/12/11/n117159...,2019-12-11 21:28:00,文正,朱立倫率青年議員成立新世代戰鬥團,【大紀元2019年12月11日訊】（大紀元記者李怡欣台灣台北報導）選戰倒數30天，韓國瑜全台...,"[朱立倫, 韓國瑜, 青年, 選區]"
7,News,大紀元,https://www.epochtimes.com/gb/19/12/11/n117160...,2019-12-11 21:23:00,文正,台國民黨拒絕反滲透法 游錫堃：藍有點紅,【大紀元2019年12月11日訊】（大紀元記者徐翠玲台灣綜合報導）對於國民黨團拒絕協商民進黨...,"[國民黨, 反滲透法, 游錫堃]"
8,News,大紀元,https://www.epochtimes.com/gb/19/12/11/n117157...,2019-12-11 21:12:00,,多家廠商憂中共觀感 時代力量黃傘竟找無廠商製作,【大紀元2019年12月11日訊】時代力量秘書長吳佩芸今天爆料，許多廠商的製造工廠在中國，顧...,"[中共, 時代力量, 吳佩芸, 黃色雨傘, 我正在推動社會進步]"
9,News,大紀元,https://www.epochtimes.com/gb/19/12/11/n117159...,2019-12-11 20:10:00,玉珍,黃國昌再爆 韓國瑜家族未繳補償金,【大紀元2019年12月11日訊】（大紀元記者吳旻洲、李怡欣台灣台北報導）時代力量立委黃國昌...,"[黃國昌, 韓國瑜, 國產署, 國有土地]"
